In [ ]:
"""======H2_cpu========="""

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from pyscf import gto, scf, fci


def get_h2_coords(d):
    """Returns coordinates for H2 molecule along z-axis"""
    return [[0, 0, -d / 2], [0, 0, d / 2]]


def get_pyscf_mol(coords, charge, spin, basis):
    atom_spec = ""
    for c in coords:
        atom_spec += f"H {c[0]} {c[1]} {c[2]}; "
    atom_spec = atom_spec.strip("; ")
    mol = gto.Mole()
    mol.atom = atom_spec
    mol.basis = basis
    mol.charge = charge
    mol.spin = spin
    mol.build()
    return mol


def get_qiskit_problem(coords, charge, spin, basis):
    atom_spec = ""
    for c in coords:
        atom_spec += f"H {c[0]} {c[1]} {c[2]}; "
    atom_spec = atom_spec.strip("; ")
    driver = PySCFDriver(atom=atom_spec, unit=DistanceUnit.ANGSTROM,
                         charge=charge, spin=spin, basis=basis)
    return driver.run()


def run_uhf_pyscf(mol):
    mf = scf.UHF(mol)
    ehf = mf.kernel()
    return ehf


def run_fci_pyscf(mol):
    mf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf.mo_coeff)
    efci, _ = cisolver.kernel()
    return efci


def run_vqe(problem, mapper):
    estimator = Estimator()
    optimizer = COBYLA(maxiter=200)
    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)
    vqe = VQE(estimator, ansatz, optimizer, initial_point=[0] * ansatz.num_parameters)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    return problem.interpret(result).total_energies[0].real


# === Main Setup ===
basis = 'sto3g'
charge = 0
spin = 0
dists = np.linspace(0.4, 4.0, 14)

vqe_energies, uhf_energies, fci_energies = [], [], []

for d in dists:
    coords = get_h2_coords(d)

    # VQE
    try:
        problem = get_qiskit_problem(coords, charge, spin, basis)
        mapper = ParityMapper(num_particles=problem.num_particles)
        vqe_energy = run_vqe(problem, mapper)
    except Exception as e:
        vqe_energy = np.nan
        print(f"VQE failed at d={d:.2f}: {e}")

    # UHF
    try:
        mol = get_pyscf_mol(coords, charge, spin, basis)
        uhf_energy = run_uhf_pyscf(mol)
    except Exception as e:
        uhf_energy = np.nan
        print(f"UHF failed at d={d:.2f}: {e}")

    # FCI
    try:
        fci_energy = run_fci_pyscf(mol)
    except Exception as e:
        fci_energy = np.nan
        print(f"FCI failed at d={d:.2f}: {e}")

    vqe_energies.append(vqe_energy)
    uhf_energies.append(uhf_energy)
    fci_energies.append(fci_energy)
    print(f"d={d:.2f} | VQE={vqe_energy:.6f} | UHF={uhf_energy:.6f} | FCI={fci_energy:.6f}")


# === Plotting ===
plt.figure(figsize=(6, 4.5))
plt.plot(dists, vqe_energies, label='VQE', color='limegreen')
plt.plot(dists, uhf_energies, label='UHF', color='red')
plt.plot(dists, fci_energies, '--', label='FCI', color='black')
plt.xlabel("H-H Distance (Å)")
plt.ylabel("Energy (Hartree)")
plt.title("H$_2$ Potential Energy Curve")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
# === Print single ground state energy ===
min_index = np.nanargmin(vqe_energies)
min_distance = dists[min_index]
min_energy = vqe_energies[min_index]

print("\n=== H2 Ground State Energy from VQE ===")
print(f"Minimum energy: {min_energy:.6f} Hartree")
print(f"At bond distance: {min_distance:.2f} Å")


In [ ]:
"""======Ne_cpu========"""

In [ ]:
import numpy as np
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

from pyscf import gto, scf, fci

# --- Classical Methods ---
def get_pyscf_mol(basis="sto3g"):
    mol = gto.Mole()
    mol.atom = "Ne 0.0 0.0 0.0"
    mol.basis = basis
    mol.charge = 0
    mol.spin = 0
    mol.build()
    return mol

def run_uhf_pyscf(mol):
    mf = scf.UHF(mol)
    return mf.kernel()

def run_fci_pyscf(mol):
    mf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf.mo_coeff)
    energy, _ = cisolver.kernel()
    return energy

# --- Qiskit Problem ---
def get_qiskit_problem(basis="sto3g"):
    driver = PySCFDriver(
        atom="Ne 0.0 0.0 0.0",
        unit=DistanceUnit.ANGSTROM,
        charge=0,
        spin=0,
        basis=basis
    )
    problem = driver.run()

    # Reduce problem size: Neon has 10 electrons, full space is heavy
    transformer = ActiveSpaceTransformer(
        num_electrons=10,
        num_spatial_orbitals=5  # Restrict to valence + near-valence
    )
    return transformer.transform(problem)

# --- VQE ---
def run_vqe_cpu(problem, mapper):
    estimator = Estimator()
    optimizer = COBYLA(maxiter=200)

    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)

    vqe = VQE(estimator, ansatz, optimizer, initial_point=[0]*ansatz.num_parameters)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    return problem.interpret(result).total_energies[0].real

# --- Main Calculation ---
try:
    problem = get_qiskit_problem()
    mapper = ParityMapper(num_particles=problem.num_particles)

    vqe_energy = run_vqe_cpu(problem, mapper)
    mol = get_pyscf_mol()
    uhf_energy = run_uhf_pyscf(mol)

    try:
        fci_energy = run_fci_pyscf(mol)
    except Exception:
        fci_energy = np.nan  # Too big for exact FCI

    print("=== Neon Ground State Energy (Hartree) ===")
    print(f"VQE: {vqe_energy:.6f}")
    print(f"UHF: {uhf_energy:.6f}")
    print(f"FCI: {fci_energy if not np.isnan(fci_energy) else 'Too large for FCI'}")

except Exception as e:
    print("Calculation failed:", e)


In [ ]:
"""======H2O======"""

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from pyscf import gto, scf, fci


# --- Classical Methods with PySCF ---
def get_pyscf_mol(coords, basis="sto3g"):
    atom_spec = ""
    for atom in coords:
        atom_spec += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
    atom_spec = atom_spec.strip("; ")

    mol = gto.Mole()
    mol.atom = atom_spec
    mol.basis = basis
    mol.charge = 0
    mol.spin = 0
    mol.build()
    return mol


def run_uhf_pyscf(mol):
    mf = scf.UHF(mol)
    return mf.kernel()


def run_fci_pyscf(mol):
    mf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf.mo_coeff)
    energy, _ = cisolver.kernel()
    return energy


# --- Qiskit Problem ---
def get_qiskit_problem(coords, basis="sto3g"):
    atom_spec = ""
    for atom in coords:
        atom_spec += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
    atom_spec = atom_spec.strip("; ")

    driver = PySCFDriver(
        atom=atom_spec,
        unit=DistanceUnit.ANGSTROM,
        charge=0,
        spin=0,
        basis=basis
    )
    problem = driver.run()

    # Active space to reduce qubit count
    transformer = ActiveSpaceTransformer(
        num_electrons=8,            # 8 valence electrons in H₂O
        num_spatial_orbitals=6      # Reasonable active space size
    )
    return transformer.transform(problem)


# --- VQE (CPU) ---
def run_vqe_cpu(problem, mapper):
    estimator = Estimator()
    optimizer = COBYLA(maxiter=200)

    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)

    vqe = VQE(estimator, ansatz, optimizer, initial_point=[0]*ansatz.num_parameters)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    return problem.interpret(result).total_energies[0].real


# --- Geometry Scan for H₂O ---
def get_h2o_geometry(bond_length):
    """
    Returns a bent H₂O geometry with variable O-H bond length.
    Angle fixed at ~104.5 degrees.
    """
    angle = 104.5 * np.pi / 180  # radians
    x = bond_length * np.sin(angle / 2)
    y = bond_length * np.cos(angle / 2)

    return [
        ("O", (0.0, 0.0, 0.0)),
        ("H", (x, y, 0.0)),
        ("H", (-x, y, 0.0))
    ]


# --- Main Loop ---
bond_lengths = np.linspace(0.7, 2.0, 8)
vqe_energies, uhf_energies, fci_energies = [], [], []

for d in bond_lengths:
    coords = get_h2o_geometry(d)

    # VQE
    try:
        problem = get_qiskit_problem(coords)
        mapper = ParityMapper(num_particles=problem.num_particles)
        vqe_energy = run_vqe_cpu(problem, mapper)
    except Exception as e:
        vqe_energy = np.nan
        print(f"VQE failed at {d:.2f} Å: {e}")

    # UHF
    try:
        mol = get_pyscf_mol(coords)
        uhf_energy = run_uhf_pyscf(mol)
    except Exception as e:
        uhf_energy = np.nan
        print(f"UHF failed at {d:.2f} Å: {e}")

    # FCI
    try:
        fci_energy = run_fci_pyscf(mol)
    except Exception as e:
        fci_energy = np.nan
        print(f"FCI failed at {d:.2f} Å: {e}")

    vqe_energies.append(vqe_energy)
    uhf_energies.append(uhf_energy)
    fci_energies.append(fci_energy)

    print(f"O-H bond {d:.2f} Å | VQE={vqe_energy:.6f} | UHF={uhf_energy:.6f} | FCI={fci_energy:.6f}")


# --- Plot ---
plt.figure(figsize=(6,4.5))
plt.plot(bond_lengths, vqe_energies, label='VQE', color='limegreen')
plt.plot(bond_lengths, uhf_energies, label='UHF', color='red')
plt.plot(bond_lengths, fci_energies, '--', label='FCI', color='black')
plt.xlabel("O-H Bond Length (Å)")
plt.ylabel("Energy (Hartree)")
plt.title("H₂O Potential Energy Curve")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# --- Minimum Energies ---
print("\n========= Minimum Energies =========")
print(f"Min VQE Energy: {np.nanmin(vqe_energies):.6f} Hartree")
print(f"Min UHF Energy: {np.nanmin(uhf_energies):.6f} Hartree")
print(f"Min FCI Energy: {np.nanmin(fci_energies):.6f} Hartree")


In [ ]:
'''=========CH4_cpu========='''


In [ ]:
import numpy as np
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

from pyscf import gto, scf, fci

# --- Classical Methods ---
def get_pyscf_mol(basis="sto3g"):
    # Approx. tetrahedral geometry of methane (CH4) in Å
    coords = """
    C 0.0000 0.0000 0.0000
    H 0.6291 0.6291 0.6291
    H -0.6291 -0.6291 0.6291
    H -0.6291 0.6291 -0.6291
    H 0.6291 -0.6291 -0.6291
    """
    mol = gto.Mole()
    mol.atom = coords
    mol.basis = basis
    mol.charge = 0
    mol.spin = 0
    mol.build()
    return mol

def run_uhf_pyscf(mol):
    mf = scf.UHF(mol)
    return mf.kernel()

def run_fci_pyscf(mol):
    mf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf.mo_coeff)
    energy, _ = cisolver.kernel()
    return energy

# --- Qiskit Problem ---
def get_qiskit_problem(basis="sto3g"):
    coords = """
    C 0.0000 0.0000 0.0000
    H 0.6291 0.6291 0.6291
    H -0.6291 -0.6291 0.6291
    H -0.6291 0.6291 -0.6291
    H 0.6291 -0.6291 -0.6291
    """
    driver = PySCFDriver(
        atom=coords,
        unit=DistanceUnit.ANGSTROM,
        charge=0,
        spin=0,
        basis=basis
    )
    problem = driver.run()

    # Reduce to active space to make VQE feasible
    transformer = ActiveSpaceTransformer(
        num_electrons=8,      # CH4 has 10 electrons; keep valence electrons
        num_spatial_orbitals=8  # Keep enough orbitals for correlation
    )
    return transformer.transform(problem)

# --- VQE ---
def run_vqe_cpu(problem, mapper):
    estimator = Estimator()
    optimizer = COBYLA(maxiter=300)

    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)

    vqe = VQE(estimator, ansatz, optimizer, initial_point=[0]*ansatz.num_parameters)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    return problem.interpret(result).total_energies[0].real

# --- Main Calculation ---
try:
    problem = get_qiskit_problem()
    mapper = ParityMapper(num_particles=problem.num_particles)

    vqe_energy = run_vqe_cpu(problem, mapper)
    mol = get_pyscf_mol()
    uhf_energy = run_uhf_pyscf(mol)

    try:
        fci_energy = run_fci_pyscf(mol)
    except Exception:
        fci_energy = np.nan  # Too big for exact FCI

    print("=== Methane (CH4) Ground State Energy (Hartree) ===")
    print(f"VQE: {vqe_energy:.6f}")
    print(f"UHF: {uhf_energy:.6f}")
    print(f"FCI: {fci_energy if not np.isnan(fci_energy) else 'Too large for FCI'}")

except Exception as e:
    print("Calculation failed:", e)


In [ ]:
"""======N2========="""

In [ ]:
# cpu_vqe_n2.py
import numpy as np
import matplotlib.pyplot as plt

from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

from qiskit.quantum_info import Statevector

# PySCF for classical references
from pyscf import gto, scf, fci

# --- User-tunable parameters ---
basis = "sto3g"
# Active space (CAS-like). Adjust if you want larger/smaller active space:
ACTIVE_ELECTRONS = 6      # e.g., 6 electrons in active space (typical minimal CAS for N2)
ACTIVE_SPATIAL_ORBITALS = 6  # number of spatial orbitals in active space
# Bond distances to scan (N - N bond in Angstrom)
dists = np.linspace(0.9, 2.6, 14)  # includes equilibrium ~1.098 Å and beyond

# --- Helpers ---
def build_pyscf_mol(distance, basis=basis):
    # linear molecule along z-axis: N at -d/2 and +d/2
    atom = f"N 0 0 {-distance/2}; N 0 0 {distance/2}"
    mol = gto.Mole()
    mol.atom = atom
    mol.basis = basis
    mol.charge = 0
    mol.spin = 0
    mol.build()
    return mol

def run_uhf_pyscf(mol):
    mf = scf.UHF(mol)
    return mf.kernel()

def run_fci_pyscf(mol):
    # Note: FCI on full molecule can be huge; we do FCI in full AO space if feasible.
    mf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf.mo_coeff)
    e, _ = cisolver.kernel()
    return e

def build_qiskit_problem(distance, basis=basis,
                        active_electrons=ACTIVE_ELECTRONS,
                        active_orbitals=ACTIVE_SPATIAL_ORBITALS):
    atom = f"N 0 0 {-distance/2}; N 0 0 {distance/2}"
    driver = PySCFDriver(atom=atom, basis=basis, unit=DistanceUnit.ANGSTROM,
                         charge=0, spin=0)
    problem = driver.run()
    # Reduce to active space to keep qubit count / cost manageable
    transformer = ActiveSpaceTransformer(num_electrons=active_electrons,
                                         num_spatial_orbitals=active_orbitals)
    problem_reduced = transformer.transform(problem)
    return problem_reduced

def exact_energy_from_qubit_op(qubit_op):
    # Dense matrix diagonalization (for the reduced operator)
    mat = qubit_op.to_matrix()
    # use eigvalsh because Hamiltonian is Hermitian
    evals = np.linalg.eigvalsh(mat)
    return float(np.min(np.real(evals)))

def run_vqe_cpu(problem, mapper, maxiter=300):
    estimator = Estimator()              # CPU estimator
    optimizer = COBYLA(maxiter=maxiter) # stable optimizer for small systems

    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper,
                   initial_state=init_state)

    vqe = VQE(estimator, ansatz, optimizer, initial_point=[0]*ansatz.num_parameters)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())

    result = vqe.compute_minimum_eigenvalue(qubit_op)
    # Interpret result to total energy (adds nuclear repulsion etc.)
    energy = problem.interpret(result).total_energies[0].real
    return energy, result, qubit_op

# --- Main sweep ---
vqe_energies = []
uhf_energies = []
exact_energies = []

for d in dists:
    print(f"\nDistance {d:.3f} Å")
    # classical pyscf references
    try:
        mol_full = build_pyscf_mol(d)
        uhf_e = run_uhf_pyscf(mol_full)
    except Exception as e:
        print("UHF failed:", e)
        uhf_e = np.nan

    # FCI on full AO space might be heavy; try but catch
    try:
        fci_e = run_fci_pyscf(mol_full)
    except Exception as e:
        # Likely too expensive; set NaN
        fci_e = np.nan

    uhf_energies.append(uhf_e)
    fci_energies = fci_e  # not used as array if too heavy

    # Qiskit problem (reduced active space)
    try:
        problem = build_qiskit_problem(d)
        mapper = ParityMapper(num_particles=problem.num_particles)
    except Exception as e:
        print("Problem build failed:", e)
        vqe_energies.append(np.nan)
        exact_energies.append(np.nan)
        continue

    # exact (dense) for reduced qubit operator
    try:
        qubit_op = mapper.map(problem.hamiltonian.second_q_op())
        exact_e = exact_energy_from_qubit_op(qubit_op)
    except Exception as e:
        print("Exact (dense) failed:", e)
        exact_e = np.nan

    exact_energies.append(exact_e)

    # run VQE on CPU
    try:
        vqe_e, vqe_result, _ = run_vqe_cpu(problem, mapper, maxiter=200)
        print(f" VQE (CPU): {vqe_e:.8f} Hartree")
    except Exception as e:
        print("VQE failed:", e)
        vqe_e = np.nan

    vqe_energies.append(vqe_e)

# --- Plotting ---
plt.figure(figsize=(7,5))
plt.plot(dists, vqe_energies, 'o-', label='VQE (active space)', color='limegreen')
plt.plot(dists, uhf_energies, 's-', label='UHF (full)', color='red')
plt.plot(dists, exact_energies, '--', label='Exact (reduced)', color='black')
plt.xlabel("N–N Distance (Å)")
plt.ylabel("Energy (Hartree)")
plt.title(f"N$_2$ Potential Energy Curve (active space: {ACTIVE_ELECTRONS}e, {ACTIVE_SPATIAL_ORBITALS}o)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# --- Report minimum from VQE sweep ---
if np.all(np.isnan(vqe_energies)):
    print("\nNo successful VQE energies calculated.")
else:
    idx = int(np.nanargmin(vqe_energies))
    print("\n=== Minimum VQE result (from sweep) ===")
    print(f"Distance = {dists[idx]:.3f} Å")
    print(f"VQE energy = {vqe_energies[idx]:.8f} Hartree")


In [ ]:
'''==========O2 cpu========='''

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

from pyscf import gto, scf, fci

# ---------------- User parameters ----------------
BASIS = "sto3g"   # keep small for laptop; change if you need higher accuracy
# Active-space (CAS) parameters — tune to fit CPU/RAM budget:
# O2 has 16 electrons (two O atoms × 8 each). A common small CAS for O2 is (6,6) or (8,8).
ACTIVE_ELECTRONS = 6
ACTIVE_SPATIAL_ORBITALS = 6

DISTANCES = np.linspace(1.0, 2.6, 13)  # O-O distances in Angstrom (includes eq ~1.21 Å)
MAXITER = 200  # VQE optimizer iterations (lower for quick tests)
# -------------------------------------------------

def build_full_pyscf_mol(distance, basis=BASIS):
    # Place O atoms on z-axis at +/- distance/2
    atom = f"O 0 0 {-distance/2}; O 0 0 {distance/2}"
    mol = gto.Mole()
    mol.atom = atom
    mol.basis = basis
    mol.charge = 0
    mol.spin = 0  # singlet/triplet selection: here we assume singlet (spin=0). Change if needed.
    mol.build()
    return mol

def run_uhf_pyscf(mol):
    mf = scf.UHF(mol)
    return mf.kernel()

def build_qiskit_problem(distance, basis=BASIS,
                        active_electrons=ACTIVE_ELECTRONS,
                        active_orbitals=ACTIVE_SPATIAL_ORBITALS):
    atom = f"O 0 0 {-distance/2}; O 0 0 {distance/2}"
    driver = PySCFDriver(atom=atom, basis=basis, unit=DistanceUnit.ANGSTROM,
                         charge=0, spin=0)
    problem = driver.run()
    # Reduce to active space to keep qubit/cost manageable
    transformer = ActiveSpaceTransformer(num_electrons=active_electrons,
                                         num_spatial_orbitals=active_orbitals)
    problem_reduced = transformer.transform(problem)
    return problem_reduced

def dense_exact_energy_from_qubit_op(qubit_op):
    # Dense diagonalization in reduced space
    mat = qubit_op.to_matrix()
    evals = np.linalg.eigvalsh(mat)
    return float(np.min(np.real(evals)))

def run_vqe_cpu(problem, mapper, maxiter=MAXITER):
    estimator = Estimator()              # CPU estimator
    optimizer = COBYLA(maxiter=maxiter) # stable optimizer for small systems

    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper,
                   initial_state=init_state)

    vqe = VQE(estimator, ansatz, optimizer, initial_point=[0]*ansatz.num_parameters)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())

    result = vqe.compute_minimum_eigenvalue(qubit_op)
    energy = problem.interpret(result).total_energies[0].real
    return energy, result, qubit_op

# --------- Sweep over bond distances ----------
vqe_energies = []
uhf_energies = []
exact_energies = []

for d in DISTANCES:
    print(f"\n--- O-O distance = {d:.3f} Å ---")

    # Classical UHF (full molecule)
    try:
        mol_full = build_full_pyscf_mol(d)
        uhf_e = run_uhf_pyscf(mol_full)
        print(f"UHF (PySCF) : {uhf_e:.8f}")
    except Exception as e:
        print("UHF failed:", e)
        uhf_e = np.nan
    uhf_energies.append(uhf_e)

    # Build reduced Qiskit problem
    try:
        problem = build_qiskit_problem(d)
        mapper = ParityMapper(num_particles=problem.num_particles)
        print(f"Reduced problem: electrons={problem.num_particles}, spatial_orbitals={problem.num_spatial_orbitals}")
    except Exception as e:
        print("Problem build failed:", e)
        vqe_energies.append(np.nan)
        exact_energies.append(np.nan)
        continue

    # Exact (dense) in reduced active space
    try:
        qubit_op = mapper.map(problem.hamiltonian.second_q_op())
        exact_e = dense_exact_energy_from_qubit_op(qubit_op)
        print(f"Exact (active space) : {exact_e:.8f}")
    except Exception as e:
        print("Exact (active space) failed:", e)
        exact_e = np.nan
    exact_energies.append(exact_e)

    # Run VQE on CPU for reduced problem
    try:
        vqe_e, vqe_result, _ = run_vqe_cpu(problem, mapper, maxiter=MAXITER)
        print(f"VQE (CPU)            : {vqe_e:.8f}")
    except Exception as e:
        print("VQE failed:", e)
        vqe_e = np.nan

    vqe_energies.append(vqe_e)

# ---------- Plot ----------
plt.figure(figsize=(7,5))
plt.plot(DISTANCES, vqe_energies, 'o-', label='VQE (active)', color='limegreen')
plt.plot(DISTANCES, uhf_energies, 's-', label='UHF (full)', color='red')
plt.plot(DISTANCES, exact_energies, '--', label='Exact (active)', color='black')
plt.xlabel("O–O Distance (Å)")
plt.ylabel("Energy (Hartree)")
plt.title(f"O$_2$ Potential Energy Curve (active: {ACTIVE_ELECTRONS}e, {ACTIVE_SPATIAL_ORBITALS}o)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ---------- Minimum VQE value ----------
if np.all(np.isnan(vqe_energies)):
    print("\nNo successful VQE energies calculated.")
else:
    idx = int(np.nanargmin(vqe_energies))
    print("\n=== Minimum VQE result (from sweep) ===")
    print(f"Distance = {DISTANCES[idx]:.3f} Å")
    print(f"VQE energy = {vqe_energies[idx]:.8f} Hartree")


In [ ]:
'''==========Ethyne (C₂H₂) cpu ========='''

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from pyscf import gto, scf


# ======================
# Molecule definitions
# ======================
def get_c2h2_coords(c_distance=1.2, ch_distance=1.06):
    """
    Returns coordinates for C2H2 in linear arrangement along z-axis.
    c_distance: C–C distance (Å)
    ch_distance: C–H distance (Å)
    """
    coords = []
    coords.append(["C", [0, 0, -c_distance/2]])           # Carbon 1
    coords.append(["C", [0, 0, c_distance/2]])            # Carbon 2
    coords.append(["H", [0, 0, -c_distance/2 - ch_distance]])  # Hydrogen 1
    coords.append(["H", [0, 0, c_distance/2 + ch_distance]])   # Hydrogen 2
    return coords


def get_pyscf_mol(coords, basis, charge=0, spin=0):
    atom_spec = "; ".join([f"{atom} {x} {y} {z}" for atom, (x, y, z) in coords])
    mol = gto.Mole()
    mol.atom = atom_spec
    mol.basis = basis
    mol.charge = charge
    mol.spin = spin
    mol.build()
    return mol


def get_qiskit_problem(coords, basis, charge=0, spin=0, active_electrons=None, active_orbitals=None):
    atom_spec = "; ".join([f"{atom} {x} {y} {z}" for atom, (x, y, z) in coords])
    driver = PySCFDriver(atom=atom_spec, unit=DistanceUnit.ANGSTROM,
                         charge=charge, spin=spin, basis=basis)
    problem = driver.run()
    if active_electrons is not None and active_orbitals is not None:
        transformer = ActiveSpaceTransformer(num_electrons=active_electrons,
                                             num_spatial_orbitals=active_orbitals)
        problem = transformer.transform(problem)
    return problem


def run_uhf_pyscf(mol):
    mf = scf.UHF(mol)
    return mf.kernel()


def run_vqe_cpu(problem, mapper, maxiter=200):
    estimator = Estimator()
    optimizer = COBYLA(maxiter=maxiter)
    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper,
                   initial_state=init_state)
    vqe = VQE(estimator, ansatz, optimizer, initial_point=[0]*ansatz.num_parameters)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    return problem.interpret(result).total_energies[0].real


# ======================
# Main parameters
# ======================
BASIS = "sto3g"
CHARGE = 0
SPIN = 0
ACTIVE_ELECTRONS = 8
ACTIVE_SPATIAL_ORBITALS = 8
CC_DISTANCES = np.linspace(1.0, 1.6, 7)  # Å, sweep over C–C distances
CH_DISTANCE = 1.06                       # Å, fixed C–H

vqe_energies = []
uhf_energies = []

for cc_d in CC_DISTANCES:
    coords = get_c2h2_coords(c_distance=cc_d, ch_distance=CH_DISTANCE)

    # UHF
    try:
        mol = get_pyscf_mol(coords, BASIS, CHARGE, SPIN)
        uhf_e = run_uhf_pyscf(mol)
    except Exception as e:
        print(f"UHF failed at CC={cc_d:.2f} Å: {e}")
        uhf_e = np.nan
    uhf_energies.append(uhf_e)

    # VQE
    try:
        problem = get_qiskit_problem(coords, BASIS, CHARGE, SPIN,
                                     active_electrons=ACTIVE_ELECTRONS,
                                     active_orbitals=ACTIVE_SPATIAL_ORBITALS)
        mapper = ParityMapper(num_particles=problem.num_particles)
        vqe_e = run_vqe_cpu(problem, mapper, maxiter=200)
    except Exception as e:
        print(f"VQE failed at CC={cc_d:.2f} Å: {e}")
        vqe_e = np.nan
    vqe_energies.append(vqe_e)

    print(f"CC={cc_d:.2f} Å | VQE={vqe_e:.6f} | UHF={uhf_e:.6f}")


# ======================
# Plotting
# ======================
plt.figure(figsize=(6, 4.5))
plt.plot(CC_DISTANCES, vqe_energies, label='VQE (active space)', color='limegreen')
plt.plot(CC_DISTANCES, uhf_energies, label='UHF (full)', color='red')
plt.xlabel("C–C Distance (Å)")
plt.ylabel("Energy (Hartree)")
plt.title("C$_2$H$_2$ (Ethyne) Potential Energy Curve")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Minimum VQE
if not np.all(np.isnan(vqe_energies)):
    idx = int(np.nanargmin(vqe_energies))
    print("\n=== Minimum VQE Energy ===")
    print(f"CC Distance = {CC_DISTANCES[idx]:.3f} Å")
    print(f"Energy = {vqe_energies[idx]:.8f} Hartree")


In [ ]:
'''=======Ethene (C₂H₄)  cpu======='''

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from pyscf import gto, scf

# ----------------- User parameters -----------------
BASIS = "sto3g"
ACTIVE_ELECTRONS = 8     # tune for performance
ACTIVE_SPATIAL_ORBITALS = 8
CC_DISTANCES = np.linspace(1.2, 1.6, 5)  # Å
CH_DISTANCE = 1.09  # Å typical C–H bond length
MAXITER = 150
# ---------------------------------------------------

def build_full_pyscf_mol(cc_distance, ch_distance=CH_DISTANCE, basis=BASIS):
    """
    Build planar ethene geometry along z-axis:
    C1 at -cc/2, C2 at +cc/2, each bonded to 2 hydrogens in-plane.
    """
    c1_z = -cc_distance / 2
    c2_z = cc_distance / 2

    # Hydrogens placed symmetrically along y-axis for planarity
    h1_y = ch_distance
    h2_y = -ch_distance

    atom = (
        f"C 0 {0} {c1_z}; "
        f"C 0 {0} {c2_z}; "
        f"H 0 {h1_y} {c1_z}; "
        f"H 0 {h2_y} {c1_z}; "
        f"H 0 {h1_y} {c2_z}; "
        f"H 0 {h2_y} {c2_z}"
    )

    mol = gto.Mole()
    mol.atom = atom
    mol.basis = basis
    mol.charge = 0
    mol.spin = 0
    mol.build()
    return mol

def build_qiskit_problem(cc_distance, ch_distance=CH_DISTANCE,
                         active_electrons=ACTIVE_ELECTRONS,
                         active_orbitals=ACTIVE_SPATIAL_ORBITALS,
                         basis=BASIS):
    c1_z = -cc_distance / 2
    c2_z = cc_distance / 2
    h1_y = ch_distance
    h2_y = -ch_distance

    atom = (
        f"C 0 {0} {c1_z}; "
        f"C 0 {0} {c2_z}; "
        f"H 0 {h1_y} {c1_z}; "
        f"H 0 {h2_y} {c1_z}; "
        f"H 0 {h1_y} {c2_z}; "
        f"H 0 {h2_y} {c2_z}"
    )

    driver = PySCFDriver(atom=atom,
                         unit=DistanceUnit.ANGSTROM,
                         basis=basis,
                         charge=0,
                         spin=0)
    problem = driver.run()

    # Active space reduction
    transformer = ActiveSpaceTransformer(num_electrons=active_electrons,
                                         num_spatial_orbitals=active_orbitals)
    problem_reduced = transformer.transform(problem)
    return problem_reduced

def run_vqe_cpu(problem, mapper, maxiter=MAXITER):
    estimator = Estimator()
    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper,
                   initial_state=init_state)
    optimizer = COBYLA(maxiter=maxiter, tol=1e-3)
    vqe = VQE(estimator, ansatz, optimizer, initial_point=[0]*ansatz.num_parameters)

    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    return problem.interpret(result).total_energies[0].real

# ----------------- Sweep -----------------
vqe_energies = []
uhf_energies = []
exact_active_energies = []

for cc in CC_DISTANCES:
    print(f"\n=== C–C distance: {cc:.3f} Å ===")

    # Classical UHF
    try:
        mol_full = build_full_pyscf_mol(cc)
        uhf_e = scf.UHF(mol_full).kernel()
        print(f"UHF (full): {uhf_e:.8f}")
    except Exception as e:
        print("UHF failed:", e)
        uhf_e = np.nan
    uhf_energies.append(uhf_e)

    # Qiskit problem
    try:
        problem = build_qiskit_problem(cc)
        mapper = ParityMapper(num_particles=problem.num_particles)
        print(f"Active space: electrons={problem.num_particles}, orbitals={problem.num_spatial_orbitals}")
    except Exception as e:
        print("Problem build failed:", e)
        vqe_energies.append(np.nan)
        exact_active_energies.append(np.nan)
        continue

    # Exact (active space)
    try:
        qubit_op = mapper.map(problem.hamiltonian.second_q_op())
        mat = qubit_op.to_matrix()
        exact_e = float(np.min(np.linalg.eigvalsh(mat)))
        print(f"Exact (active): {exact_e:.8f}")
    except Exception as e:
        print("Exact failed:", e)
        exact_e = np.nan
    exact_active_energies.append(exact_e)

    # VQE CPU
    try:
        vqe_e = run_vqe_cpu(problem, mapper)
        print(f"VQE (CPU): {vqe_e:.8f}")
    except Exception as e:
        print("VQE failed:", e)
        vqe_e = np.nan
    vqe_energies.append(vqe_e)

# ----------------- Plot -----------------
plt.figure(figsize=(7,5))
plt.plot(CC_DISTANCES, vqe_energies, 'o-', label='VQE (CPU, active)', color='limegreen')
plt.plot(CC_DISTANCES, uhf_energies, 's-', label='UHF (full)', color='red')
plt.plot(CC_DISTANCES, exact_active_energies, '--', label='Exact (active)', color='black')
plt.xlabel("C–C Distance (Å)")
plt.ylabel("Energy (Hartree)")
plt.title(f"C$_2$H$_4$ (Ethene) Potential Energy Curve (active: {ACTIVE_ELECTRONS}e, {ACTIVE_SPATIAL_ORBITALS}o)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ----------------- Minimum energy -----------------
if not all(np.isnan(vqe_energies)):
    idx = int(np.nanargmin(vqe_energies))
    print("\n=== Minimum VQE (CPU) result from sweep ===")
    print(f"C–C distance = {CC_DISTANCES[idx]:.3f} Å")
    print(f"VQE (CPU) energy = {vqe_energies[idx]:.8f} Hartree")
else:
    print("No successful VQE points to report.")
